In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [23]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1403,tt0372532,15000000,47175038,The Wedding Date,Debra Messing|Dermot Mulroney|Amy Adams|Jack D...,Clare Kilner,Love doesn't come cheap.,With the wedding of her younger sister fast ap...,88,Comedy|Romance,Universal Pictures|26 Films,2/4/2005,6.2,2005
1584,tt0362227,60000000,219417255,The Terminal,Tom Hanks|Catherine Zeta-Jones|Stanley Tucci|C...,Steven Spielberg,Life is waiting.,Viktor Navorski is a man without a country; hi...,128,Comedy|Drama,DreamWorks SKG|Amblin Entertainment|Parkes/Mac...,6/17/2004,7.0,2004
679,tt0462465,50000000,7033683,Outlander,Jim Caviezel|Sophia Myles|Jack Huston|Ron Perl...,Howard McCain,It destroyed his world. He won't let it destro...,"During the reign of the Vikings, a man from an...",115,Fantasy|Action|Science Fiction,Ascendant Pictures|Rising Star|VIP Medienfonds...,4/24/2008,6.0,2008
1352,tt0418763,72000000,96889998,Jarhead,Jamie Foxx|Jake Gyllenhaal|Scott MacDonald|Luc...,Sam Mendes,Welcome to the suck.,Jarhead is a film about a US Marine Anthony Sw...,125,Drama|War,Universal Pictures|Neal Street Productions|Red...,11/4/2005,6.4,2005
1673,tt0477348,25000000,171600000,No Country for Old Men,Tommy Lee Jones|Javier Bardem|Josh Brolin|Wood...,Joel Coen|Ethan Coen,There are no clean getaways.,"Llewelyn Moss stumbles upon dead bodies, $2 mi...",122,Crime|Drama|Thriller,Miramax Films|Scott Rudin Productions|Paramoun...,11/8/2007,7.6,2007


In [24]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Пользовательсике функции

In [25]:
# счетчик уникальных значений в массиве при учете того, что разделитель "|"
def un_name_count (df):
    names={} # создание словаря из имен (ключ) и подсчета количества уникальных значений
    for i in df.index:
        names_str=list(df[i].split('|'))
        for j in names_str:
            if j in names.keys():
                names[j]+=1 # если имя уже есть в словаре, то увеличиваем счетчик
            else:
                names[j]=1 # если имени нет в словаре, начинаем счетчик и создаем ключ
    return names # возвращаем словарь по результаам функции

# подсчет прибыльных и убыточных жанров
# names - перечень уникальных значений, names_list - список состоящий из наборов names, profit -
# массив разницы между сборами и бюджетом
def profit_count(names,names_list,profit):
    names_dict=dict.fromkeys(names)

    for i in names_dict.keys():
        names_dict[i] = [0 ,0]

    for i in names_list.index:
        for j in names:
            if j in names_list[i]:
                # если прибыль больше 0 увеличивает счетчик "прибыльности". если меньше 0, счетчик убыточности
                if profit.iloc[i]>0:
                    names_dict[j][0]+=1
                else:
                    names_dict[j][1] += 1

    return names_dict

# функия для суммирования значений
# names - перечень уникальных значений, names_list - список состоящий из наборов names,
# values - массив значений по которым выполняется суммирование
def summ_values(names,names_list,values):
    names_dict = dict.fromkeys(names)

    for i in names_dict.keys():
        names_dict[i] = 0

    for i in names_list.index:
        for j in names:
            if j in names_list[i]:
                # проверка на тип. если строка, считать количество. В остальных случаях выполнять суммирование.
                # Подразумевается, что входными данными могут быть int, float или str
                if type(values[i]) is not str:
                    names_dict[j]+=values[i]
                else:
                    names_dict[j]+=1

    return names_dict


#функция для подсчета вхождения значений списка names в серии names_list и значения s_name в серии s_name_list
# подразумевается, что размерности s_name_list и names_list одинаковые и формирующиеся из одногои тогоже DF
def count_smt(names,s_name,names_list,s_name_list):
    names_dict = dict.fromkeys(names)

    for i in names_dict.keys():
        names_dict[i] = 0

    for i in names_list.index:
        for j in names:
            if (j in names_list[i]) and (s_name in s_name_list[i]):
                names_dict[j]+=1
    return names_dict

# функция приведения дат к единому формату dd.mm.yyyy
def date_standart(df):
    for i in df.index:
        if '/' in df[i]:
            date=list(df[i].split('/'))
            if len(date[0])==1: #перевод месяца к формату mm
                date[0]='0'+date[0]
            df[i]=date[1]+'.'+date[0]+'.'+date[2]
    return df

#функция для подсчета длины строки и количества слов
def mid_len(names,names_list,str_list,meth='str'):
    names_dict = dict.fromkeys(names)
    for i in names_dict.keys():
        names_dict[i] = 0

    if meth=='str': # в случае если нужно посчитать количество элементов в строке
        for i in names_list.index:
            for j in names:
                # print(names_list[i])
                # print(names_list.iloc[i])
                if (j in list(names_list[i].split('|'))):
                    names_dict[j]+=len(str_list[i].replace(' ',''))

    elif meth=='word': # в случае если нужно посчитать количество слов в строке
        for i in names_list.index:
            for j in names:
                if (j in list(names_list[i].split('|'))):
                    names_dict[j]+=len(list(str_list[i].split()))


    return names_dict

# функция для подсчета частоты вхождения элементов names в names_list
def friends(names,names_list):
    l_names = len(names) # что бы несколько раз не высичлять, маленькая "оптимизация" )

    # создания DF размеров l_names х l_names изначально заполненой нулями
    names_df = pd.DataFrame([[0]*l_names]*l_names
                            ,columns=names
                            ,index=names)

    for i in names_list:
        tmp_list=list(i.split('|')) #выделение отдельных элементов из строки

        # увеличение каждого элемента массива на 1 как индикатора, что элементы tmp_list
        # входят в общий DF на конкретные позиции. при этом исключается элемент, замкнутый на себя
        for j in tmp_list:
            for k in tmp_list:
                if j != k:
                    names_df.loc[j][k]+=1
    return names_df

# Предобработка

In [26]:
answers=dict.fromkeys('Ответ на вопрос '+str(i) for i in range(1,28)) # создадим словарь для ответов
i=1

# тут другие ваши предобработки колонок например:
data['release_date']=date_standart(data['release_date']) # перевод всех дат в тип dd.mm.yyyy
data['month']=data['release_date'].str.split('.').str[-2] # создание колонки месяца выпуска
data['title_len']=data['original_title'].str.replace(' ','').str.len() # создание колонки с длиной названия фильма
data['profit']=data.revenue-data.budget # создание колонки "прибыль"

# создание отдельного DF для жанров
genres_df=pd.DataFrame.from_dict(un_name_count(data.genres), orient='index')
genres_df.columns=['counts']
genres_df[['profit','not_profit']]=pd.DataFrame.from_dict(profit_count(genres_df.index,data.genres,data.profit)
                                                          ,orient='index')
#Создание DF для режисеров
director_df=pd.DataFrame.from_dict(un_name_count(data.director), orient='index')
director_df.columns=['counts']
director_df['summ_revenue']=pd.DataFrame.from_dict(summ_values(director_df.index,data.director,data.revenue)
                                                   , orient='index')

# создание DF для актеров
actors_df=pd.DataFrame.from_dict(un_name_count(data.cast), orient='index')
actors_df.columns=['counts']
actors_df['summ_revenue_2012']=pd.DataFrame.from_dict(summ_values(actors_df.index,data[data.release_year==2012].cast
                                      ,data[data.release_year==2012].revenue), orient='index')

# создание Series для студии Paramount Pictures
pmp_df=data[data.production_companies.str.contains('Paramount Pictures')]

# создание DF для студии
prod_comp_df=pd.DataFrame.from_dict(un_name_count(data.production_companies), orient='index')
prod_comp_df.columns=['counts']

<ipython-input-25-a0ba34b9e877>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i]=date[1]+'.'+date[0]+'.'+date[2]


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [27]:
answers['Ответ на вопрос '+str(i)]=data[data.budget==data.budget.max()].original_title
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 1 найден:  723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object


# 2. Какой из фильмов самый длительный (в минутах)?

In [28]:
answers['Ответ на вопрос '+str(i)]=data[data.runtime==data.runtime.max()].original_title
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 2 найден:  1157    Gods and Generals
Name: original_title, dtype: object


# 3. Какой из фильмов самый короткий (в минутах)?





In [29]:
answers['Ответ на вопрос '+str(i)]=data[data.runtime==data.runtime.min()].original_title
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 3 найден:  768    Winnie the Pooh
Name: original_title, dtype: object


# 4. Какова средняя длительность фильмов?


In [30]:
answers['Ответ на вопрос '+str(i)]=round(data.runtime.mean())
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 4 найден:  110


# 5. Каково медианное значение длительности фильмов? 

In [31]:
answers['Ответ на вопрос '+str(i)]=round(data.runtime.median())
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 5 найден:  107


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [32]:
answers['Ответ на вопрос '+str(i)]=data[data.profit==data.profit.max()].original_title
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 6 найден:  239    Avatar
Name: original_title, dtype: object


# 7. Какой фильм самый убыточный? 

In [33]:
answers['Ответ на вопрос '+str(i)]=data[data.profit==data.profit.min()].original_title
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 7 найден:  1245    The Lone Ranger
Name: original_title, dtype: object


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [34]:
answers['Ответ на вопрос '+str(i)]=data[data.revenue>data.budget].original_title.count()
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 8 найден:  1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [35]:
answers['Ответ на вопрос '+str(i)]=data.iloc[data[data.release_year==2008].revenue.sort_values(ascending=False).head(1)
    .index].original_title
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 9 найден:  599    The Dark Knight
Name: original_title, dtype: object


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [36]:
tmp_df=data[(data.release_year>=2012)&(data.release_year<=2014)]
answers['Ответ на вопрос '+str(i)]=tmp_df[tmp_df.profit==tmp_df.profit.min()].original_title
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 10 найден:  1245    The Lone Ranger
Name: original_title, dtype: object


# 11. Какого жанра фильмов больше всего?

In [37]:
answers['Ответ на вопрос '+str(i)]=genres_df[genres_df.counts==genres_df.counts.max()].counts
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 11 найден:  Drama    782
Name: counts, dtype: int64


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [38]:
answers['Ответ на вопрос '+str(i)]=genres_df[genres_df.profit==genres_df.profit.max()].profit
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 12 найден:  Drama    560
Name: profit, dtype: int64


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [40]:
answers['Ответ на вопрос '+str(i)]=director_df[director_df.summ_revenue==director_df.summ_revenue.max()].summ_revenue
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 13 найден:  Peter Jackson    6490593685
Name: summ_revenue, dtype: int64


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [41]:
director_df['Act_count']=pd.DataFrame.from_dict(count_smt(director_df.index,'Action',data.director,data.genres)
                                                , orient='index')
answers['Ответ на вопрос '+str(i)]=director_df[director_df.Act_count==director_df.Act_count.max()].Act_count
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 14 найден:  Robert Rodriguez    9
Name: Act_count, dtype: int64


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [42]:
answers['Ответ на вопрос '+str(i)]=actors_df[actors_df.summ_revenue_2012==actors_df.summ_revenue_2012.max()]\
    .summ_revenue_2012
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 15 найден:  Chris Hemsworth    2027450773
Name: summ_revenue_2012, dtype: int64


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
actors_df['hight_budget_count']=pd.DataFrame.from_dict(summ_values(actors_df.index,data[data.budget>=data.budget.mean()].cast
                                      ,data[data.budget>=data.budget.mean()].original_title), orient='index')
answers['Ответ на вопрос '+str(i)]=actors_df[actors_df.hight_budget_count==actors_df.hight_budget_count.max()].hight_budget_count
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 16 найден:  Matt Damon    18
Name: hight_budget_count, dtype: int64


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [44]:
genres_df['NC']=pd.DataFrame.from_dict(count_smt(genres_df.index,'Nicolas Cage',data.genres,data.cast)
                                                , orient='index')
answers['Ответ на вопрос '+str(i)]=genres_df[genres_df.NC==genres_df.NC.max()].NC
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 17 найден:  Action    17
Name: NC, dtype: int64


# 18. Самый убыточный фильм от Paramount Pictures

In [45]:
answers['Ответ на вопрос '+str(i)]=pmp_df[pmp_df.profit==pmp_df.profit.min()].original_title
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 18 найден:  925    K-19: The Widowmaker
Name: original_title, dtype: object


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [46]:
best_year=data.groupby('release_year')['revenue'].sum()
answers['Ответ на вопрос '+str(i)]=best_year[best_year==best_year.max()]
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 19 найден:  release_year
2015    25449202382
Name: revenue, dtype: int64


# 20. Какой самый прибыльный год для студии Warner Bros?

In [47]:
best_year_wb=data[data.production_companies.str.contains('Warner Bros')].groupby('release_year')['profit'].sum()
answers['Ответ на вопрос '+str(i)]=best_year_wb[best_year_wb==best_year_wb.max()]
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 20 найден:  release_year
2014    2295464519
Name: profit, dtype: int64


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [48]:
best_month=data.groupby('month')['imdb_id'].count()
answers['Ответ на вопрос '+str(i)]=best_month[best_month==best_month.max()]
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 21 найден:  month
09    227
Name: imdb_id, dtype: int64


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [50]:
i-=1
answers['Ответ на вопрос '+str(i)]=best_month[['06','07','08']].sum()
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 21 найден:  450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [51]:
director_df['films_count']=pd.DataFrame.from_dict(summ_values(director_df.index,data[(data.month=='12')
                                                                                     |(data.month=='01')
                                                                                     |(data.month=='02')]
                                        .director,data[(data.month=='12')|(data.month=='01')|(data.month=='02')]
                                        .original_title), orient='index')
answers['Ответ на вопрос '+str(i)]=director_df[director_df.films_count==director_df.films_count.max()].films_count
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 22 найден:  Peter Jackson    7
Name: films_count, dtype: int64


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [52]:
tmp_df=pd.DataFrame.from_dict(mid_len(prod_comp_df.index,data.production_companies,data.original_title), orient='index')
prod_comp_df['mid_title_len']=[tmp_df.iloc[i][0]/prod_comp_df.iloc[i].counts for i in range(len(tmp_df))]
answers['Ответ на вопрос '+str(i)]=prod_comp_df[(prod_comp_df.mid_title_len==prod_comp_df.mid_title_len.max())]\
    .mid_title_len
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 23 найден:  Four By Two Productions    72.0
Name: mid_title_len, dtype: float64


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [53]:
tmp_df=pd.DataFrame.from_dict(mid_len(prod_comp_df.index,data.production_companies,data.overview,'word'), orient='index')
prod_comp_df['mid_overview_words']=[tmp_df.iloc[i][0]/prod_comp_df.iloc[i].counts for i in range(len(tmp_df))]
answers['Ответ на вопрос '+str(i)]=prod_comp_df[(prod_comp_df.mid_overview_words==prod_comp_df.mid_overview_words.max())]\
    .mid_overview_words
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 24 найден:  Midnight Picture Show    175.0
Name: mid_overview_words, dtype: float64


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [54]:
answers['Ответ на вопрос '+str(i)]=data[['original_title','vote_average']].sort_values(['vote_average']
                                                                            ,ascending=False,).head(int(len(data)*0.01))
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])
i+=1

ответ на Вопрос № 25 найден:                                       original_title  vote_average
599                                 The Dark Knight           8.1
118                                    Interstellar           8.0
125                              The Imitation Game           8.0
9                                        Inside Out           8.0
34                                             Room           8.0
1183                        The Wolf of Wall Street           7.9
128                                       Gone Girl           7.9
1191                               12 Years a Slave           7.9
119                         Guardians of the Galaxy           7.9
1081  The Lord of the Rings: The Return of the King           7.9
1800                                        Memento           7.9
370                                       Inception           7.9
872                                     The Pianist           7.9
138                        The Grand Budapest 

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [55]:
tmp_df=friends(actors_df.index,data.cast)
tmp_df['max_counts']=tmp_df.max(axis='index')
answers['Ответ на вопрос '+str(i)]=tmp_df[tmp_df.max_counts==tmp_df.max_counts.max()].index
print(f'ответ на Вопрос № {i} найден: ',answers['Ответ на вопрос '+str(i)])

ответ на Вопрос № 26 найден:  Index(['Daniel Radcliffe', 'Emma Watson', 'Rupert Grint'], dtype='object')


In [56]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(answers)

{'Ответ на вопрос 1': 723    Pirates of the Caribbean: On Stranger Tides
 Name: original_title, dtype: object,
 'Ответ на вопрос 2': 1157    Gods and Generals
 Name: original_title, dtype: object,
 'Ответ на вопрос 3': 768    Winnie the Pooh
 Name: original_title, dtype: object,
 'Ответ на вопрос 4': 110,
 'Ответ на вопрос 5': 107,
 'Ответ на вопрос 6': 239    Avatar
 Name: original_title, dtype: object,
 'Ответ на вопрос 7': 1245    The Lone Ranger
 Name: original_title, dtype: object,
 'Ответ на вопрос 8': 1478,
 'Ответ на вопрос 9': 599    The Dark Knight
 Name: original_title, dtype: object,
 'Ответ на вопрос 10': 1245    The Lone Ranger
 Name: original_title, dtype: object,
 'Ответ на вопрос 11': Drama    782
 Name: counts, dtype: int64,
 'Ответ на вопрос 12': Drama    560
 Name: profit, dtype: int64,
 'Ответ на вопрос 13': Peter Jackson    6490593685
 Name: summ_revenue, dtype: int64,
 'Ответ на вопрос 14': Robert Rodriguez    9
 Name: Act_count, dtype: int64,
 'Ответ на вопрос 1